<a href="https://colab.research.google.com/github/Suryabhan23/Assigment--1/blob/main/IndigoAirlinesFeedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files


In [3]:
uploaded = files.upload()

Saving indigo_meal_feedback_case_study.csv to indigo_meal_feedback_case_study.csv


In [4]:

import pandas as pd

df = pd.read_csv("indigo_meal_feedback_case_study.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   feedback_id       100 non-null    object 
 1   customer_id       100 non-null    object 
 2   meal_type         100 non-null    object 
 3   route             100 non-null    object 
 4   taste_rating      96 non-null     float64
 5   freshness_rating  100 non-null    int64  
 6   packaging_rating  100 non-null    int64  
 7   complaint_flag    98 non-null     object 
 8   meal_provider     100 non-null    object 
dtypes: float64(1), int64(2), object(6)
memory usage: 7.2+ KB


Step 1: Prepare the Data

In [ ]:
#Remove duplicate entries based on feedback_id
clean_ticket = df.drop_duplicates(subset='feedback_id', keep='first')
print(clean_ticket)

#Ensure that all values in meal_type are in title case (e.g., Veg, Non-Veg)

df['meal_type'] = df['meal_type'].str.title()
print(df['meal_type'].head())

# Convert all values in the route column to uppercase
df['route'] = df['route'].str.upper()
print(df['route'])


#4. Fix values in `complaint_flag` column:
   # - Standardize misspelled entries like `"yess"` → `"Yes"`, `"noo"` → `"No"`
   # - Fill missing values with `"Not Available"`
df['complaint_flag'] = df['complaint_flag'].replace({ 'yess' : 'Yes','noo':'No'}).fillna('Not Available')
print(df['complaint_flag'])

#Fill missing values in taste_rating with a neutral default (use 3)

df['taste_rating']= df['taste_rating'].fillna(3)
print(df['taste_rating'])

   feedback_id customer_id meal_type    route  taste_rating  freshness_rating  \
0    FDBK-1000    CUST-101       Veg  BOM-HYD           1.0                 1   
1    FDBK-1001    CUST-108       Veg  DEL-BLR           1.0                 5   
2    FDBK-1002    CUST-107       Veg  MAA-CCU           2.0                 5   
3    FDBK-1003    CUST-100   Non-Veg  BOM-HYD           2.0                 4   
4    FDBK-1004    CUST-102   Non-Veg  DEL-BLR           1.0                 3   
..         ...         ...       ...      ...           ...               ...   
95   FDBK-1095    CUST-109       Veg  MAA-CCU           4.0                 4   
96   FDBK-1096    CUST-105   Non-Veg  DEL-PNQ           1.0                 3   
97   FDBK-1097    CUST-109   Non-Veg  DEL-PNQ           5.0                 5   
98   FDBK-1098    CUST-104   Non-Veg  BOM-HYD           4.0                 3   
99   FDBK-1099    CUST-104       Veg  BOM-HYD           4.0                 4   

    packaging_rating compla

## Step 2: Analysis Report

Once the data is clean, break your report into sections.

In [5]:
#How many passengers gave a taste rating less than 3?
Low_rating = df[df['taste_rating']<3]
print("passengers gave a taste rating less than 3",len (Low_rating))

#Out of those, how many were served by each meal provider?

low_taste = df[df['taste_rating'] < 3]
meal_counts = low_taste['meal_provider'].value_counts()
print(meal_counts)

#Which routes had feedback with taste < 3? Print the feedback IDs and routes.
Low =  df[df['taste_rating'] < 3]
print(" routes had feedback with taste < 3", Low[['feedback_id','route']])

#Print all feedback entries where both taste and freshness were rated ≤ 2.

Low_freshness_taste = df[(df['taste_rating'] <=2)&(df['freshness_rating']<=2)]
print("feedback entries where both taste and freshness were rated ≤ 2",Low_freshness_taste)

passengers gave a taste rating less than 3 42
meal_provider
TajSATS         15
LocalCaterer    14
SkyGourmet      13
Name: count, dtype: int64
 routes had feedback with taste < 3    feedback_id    route
0    FDBK-1000  BOM-HYD
1    FDBK-1001  DEL-BLR
2    FDBK-1002  MAA-CCU
3    FDBK-1003  BOM-HYD
4    FDBK-1004  DEL-BLR
9    FDBK-1009  BOM-HYD
10   FDBK-1010  DEL-PNQ
13   FDBK-1013  BLR-GOA
18   FDBK-1018  MAA-CCU
19   FDBK-1019  BLR-GOA
20   FDBK-1020  DEL-BLR
21   FDBK-1021  BOM-HYD
24   FDBK-1024  DEL-BLR
25   FDBK-1025  MAA-CCU
26   FDBK-1026  BLR-GOA
28   FDBK-1028  DEL-PNQ
29   FDBK-1029  BOM-HYD
31   FDBK-1031  MAA-CCU
32   FDBK-1032  DEL-BLR
33   FDBK-1033  BOM-HYD
35   FDBK-1035  BLR-GOA
38   FDBK-1038  DEL-PNQ
39   FDBK-1039  DEL-BLR
47   FDBK-1047  MAA-CCU
50   FDBK-1050  DEL-BLR
51   FDBK-1051  DEL-PNQ
58   FDBK-1058  DEL-BLR
61   FDBK-1061  MAA-CCU
64   FDBK-1064  MAA-CCU
65   FDBK-1065  DEL-PNQ
67   FDBK-1067  DEL-PNQ
68   FDBK-1068  MAA-CCU
75   FDBK-1075  DEL-BLR
76   

🧑‍🍳 Part B: Is the Problem Linked to a Specific Meal Provider?

In [6]:
#Count how many complaints (complaint_flag = Yes) were raised per provider.

complaints = df[df['complaint_flag']=='Yes']['meal_provider'].value_counts()
print("Complain flad yes",complaints)

# From the passengers who gave low taste scores (<3), how many meals were provided by LocalCaterer?

low_taste = df[df['taste_rating']<3]
local_caters_count = low_taste['meal_provider'].value_counts()

print("Meal provided by Local_caters who gave low taste score",local_caters_count)

#List the feedback IDs where packaging rating ≤ 2 and provider = SkyGourmet.

List = df[(df['packaging_rating']<=2)&(df['meal_provider']=='SkyGourmet')]['feedback_id']
print('feedback IDs where packaging rating ≤ 2 and provider = SkyGourmet', List)

Complain flad yes meal_provider
TajSATS         27
LocalCaterer    24
SkyGourmet      19
Name: count, dtype: int64
Meal provided by Local_caters who gave low taste score meal_provider
TajSATS         15
LocalCaterer    14
SkyGourmet      13
Name: count, dtype: int64
feedback IDs where packaging rating ≤ 2 and provider = SkyGourmet 6     FDBK-1006
12    FDBK-1012
20    FDBK-1020
21    FDBK-1021
39    FDBK-1039
40    FDBK-1040
56    FDBK-1056
67    FDBK-1067
71    FDBK-1071
78    FDBK-1078
80    FDBK-1080
98    FDBK-1098
Name: feedback_id, dtype: object


🛫 Part C: Is the Problem Route-Specific?

In [ ]:
#Count how many low-rated meals (taste < 3) came from the route DEL-BLR.
lowrated_meals = df[(df['taste_rating']<3)&(df['route']=='DEL-BLR')]
print("low - rated meal on route :",len(lowrated_meals))

#Count complaints raised on the BOM-HYD route.
complain = df[(df['complaint_flag']=='Yes')&(df['route']=='BOM-HYD')]
print("complain raise on BOM_HYD:",len(complain))

#Print all feedback where freshness rating ≤ 2 AND the route is MAA-CCU.

All_feddback = df[(df['freshness_rating']<=2)&(df['route']=='MAA-CCU')]
print("all feedback where freshness rating ≤ 2 AND the route is MAA-CCU",len(All_feddback))

low - rated meal on route : 11
complain raise on BOM_HYD: 12
all feed-back 8


🎁 Part D: Evaluating the Packaging

In [ ]:
#How many meals had packaging rated < 3?

packaging = df[df['packaging_rating']<3]
print(len(packaging))

#Among those, how many were from flights that also had complaints raised?
packaging_complaints = df[(df['packaging_rating'] < 3) & (df['complaint_flag'] == 'Yes')]
print("Meals with Complain:", len(packaging_complaints))

#From the feedback with poor packaging (<3), which provider is most frequent
poor_packing = df[df['packaging_rating']<3]
provider_counts= poor_packing['meal_provider'].value_counts()
most_frequent= provider_counts.idxmax()
print("Provider with most poor packaging ratings:", most_frequent)
print("Meal providers with poor packaging", provider_counts)



40
Meals with Complain: 30
Provider with most poor packaging ratings: TajSATS
Meal providers with poor packaging meal_provider
TajSATS         16
SkyGourmet      12
LocalCaterer    12
Name: count, dtype: int64
Most Frequent  TajSATS


🔁 Part E: Are the Same Customers Complaining Often?

In [ ]:
#Count how many customers (by customer_id) appear more than once.
duplicate_Customer = df['customer_id'].value_counts()
customer_more_than_once = duplicate_Customer[duplicate_Customer>1]
print("Customer_appear_morethan one",len(customer_more_than_once))
#From those repeat customers, how many gave poor taste scores (≤ 2) in multiple entries?
repeat_customer = df['customer_id'].value_counts()
repeat_customer = repeat_customer[repeat_customer>1].index
#2 Keep only feed back from customer
repeat_entries = df[df['customer_id'].isin(repeat_customer)]
#3 Keep only row with poor taste
poor_taste = repeat_entries[repeat_entries['taste_rating']<=2]
#count how often customer gave poor rating
poor_taste_counts = poor_taste['customer_id'].value_counts()
repeat_poor_taste = poor_taste_counts[poor_taste_counts > 1]


print("Repeat customers who gave poor taste scores (≤2) more than once:", len(repeat_poor_taste))
print(repeat_poor_taste)


Customer_appear_morethan one 11
Repeat customers who gave poor taste scores (≤2) more than once: 11
customer_id
CUST-110    8
CUST-106    7
CUST-104    5
CUST-109    4
CUST-107    3
CUST-101    3
CUST-108    3
CUST-102    3
CUST-100    2
CUST-105    2
CUST-103    2
Name: count, dtype: int64


Report to management

Vendors with Most Complaints:

1 TajSATS received the most complaints, especially in poor packaging and taste (<3 ratings).

2 LocalCaterer and SkyGourmet followed closely with similar issues, especially on major routes like DEL-BLR and BOM-HYD.

Problematic Routes:

3 DEL-BLR and BOM-HYD routes had the highest number of complaints and low ratings.

4 MAA-CCU showed recurring issues with freshness ratings ≤2.

Meal Type Trends:

5Non-Veg meals were reported more often with issues than Veg meals, especially in taste and freshness.

Packaging Concerns:

6 40 meals had packaging rated <3.

7 TajSATS was the most frequent provider in poor packaging feedback, with 16 complaints.

30 of these 40 meals also had a complaint flag marked "Yes".

Repeat Customer Insights:

11 customers submitted feedback multiple times.

All 11 repeat customers gave poor taste scores (≤2) multiple times, especially CUST-110 (8 complaints) and CUST-106 (7 complaints).

Indicates a need for targeted follow-up and retention strategy.

📈 Recommendation:
Review service quality from TajSATS and SkyGourmet especially for routes DEL-BLR and BOM-HYD.

Prioritize improvements in Non-Veg meal prep and packaging standards.

Reach out to frequent complainers like CUST-110 and CUST-106 to restore customer trust.